kaggleコンペ初参戦の初心者が、初めて投稿したコードです。精度は全然まだまだですが、私と同じような初心者の方が、大体どんな感じにコードを書いていけばいいのかのイメージをつかむきっかけになれば幸いです。
以下コメントでは、とにかく自分が作った特徴量で一回提出したい初心者の方向けに、どこをどういじればよいかを記載しています。（アルゴリズムはLGBを使ってます）

In [ ]:
from pyarrow import parquet as pq
import pandas as pd
import os
from sklearn.metrics import r2_score
import glob
import re
import matplotlib.pyplot as plt
import numpy as np

**以下、要編集。**trainの場合、train.csvのdataframeを引数にとって、それに生成した特徴量列を結合し、結合した関数を返すように作る（row_idを作成し、row_idで結合させる）。testの場合、ただ作成した特徴量列を持つdataframeを返すように作る。

In [ ]:
def std_ask_bid_price_and_size(df_train=None, mode='train'):
    feature_list=["ask_price1", "ask_price2", "bid_price1", "bid_price2", "ask_size1", "ask_size2", "bid_size1", "bid_size2"]
    #statistic_list=["mean","median","std","count","sum"]
    if mode == 'train':
        list_order_book_file = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*')
    else:
        list_order_book_file = glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_test.parquet/*')        

    
    for kkk, iii in enumerate(list_order_book_file):
        df_order=pd.read_parquet(iii)
        #df_order_feature = df_order.groupby("time_id")[feature_list].agg(statistic_list).reset_index()
        df_order_feature = df_order.groupby("time_id")[feature_list].agg(ask_size1_std=('ask_size1', 'std'),ask_size2_std=('ask_size2', 'std'),bid_size1_std=('bid_size1', 'std'),bid_size2_std=('bid_size2', 'std'),ask_price1_std=('ask_price1', 'std'),ask_price2_std=('ask_price2', 'std'),bid_price1_std=('bid_price1', 'std'),bid_price2_std=('bid_price2', 'std')).reset_index()
        result = re.search(r'=\d+$', iii)
        stock_id_plus_equal=result.group()
        stock_id=stock_id_plus_equal.strip("=")
        df_order_feature['stock_id']=int(stock_id)
        #df_order_feature_target = pd.merge(df_order_feature,df_train,on=['stock_id', 'time_id'],how='left')
        
        if kkk ==0:
            df_order_feature_all=df_order_feature
        else:
            df_order_feature_all=pd.concat([df_order_feature_all, df_order_feature])
    
    df_order_feature_all['row_id'] = df_order_feature_all['stock_id'].astype(str) + '-' + df_order_feature_all['time_id'].astype(str)
    df_order_feature_all=df_order_feature_all.drop(columns=["time_id", "stock_id"])
    #df_order_feature_all.drop(columns=["time_id", "stock_id"])
    if mode == 'train':
        df_joined = df_train.merge(df_order_feature_all, on =['row_id'], how = 'left')
    else:
        df_joined=df_order_feature_all
    
    return df_joined

いじる必要なし

In [ ]:
train=pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')

train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]


**以下、要編集。**train用のデータを作成する。自分が作った特徴量生成関数に、train.csvのdfを渡して、特徴量とtargetをもつdataframeを受け取る

In [ ]:
train=std_ask_bid_price_and_size(df_train=train, mode='train')
train.head()

以下、いじる必要なし

In [ ]:
import lightgbm as lgb

model = lgb.LGBMRegressor(objective='regression', learning_rate=0.1, n_estimators=1000,
                          max_depth=5, min_child_weight=1.0, colsample_bytree=0.8,
                          subsample=0.8, reg_alpha=0.0, reg_lambda=1.0)

**以下、要編集。**上で生成した、特徴量とtargetを持つdataframeから、train_x（特徴量のみのdataframe）とtrain_y（targetのみのdataframe）を作成

In [ ]:
feature_columns=['ask_price1_std', 'ask_price2_std','bid_price1_std', 'bid_price2_std']


train_x=train[feature_columns]
train_y=train['target']

**以下要編集。**light_gbmモデルにデータを渡して学習。僕はx,yともに対数をとって学習させたので、そうさせたくない場合はnp.logを取り除いてください

In [ ]:
model.fit(np.log(train_x.values), np.log(train_y.values))

**以下、要編集。**テスト用のデータを作成する。自分が作った特徴量作成関数を使って、特徴量のdataframeを取得する

In [ ]:
test=std_ask_bid_price_and_size(mode='test')
test.head()

**以下、要編集。**trainの時に使用した特徴量に絞り、データをモデルに渡して推定結果を取得する。対数をとって学習させてない場合はnp.logを取り除いてください

In [ ]:
test_x=test[feature_columns]
test_pred=model.predict(np.log(test_x.values))

**以下、要編集。**row_idと推定結果のdataframeを作成しCSV化。対数とってない場合は'target'のnp.expを取り除いてください。

In [ ]:

df_submit=pd.DataFrame({'row_id':test['row_id'],
                       'target':np.exp(test_pred)})
df_submit.to_csv("submission.csv", index=False)